# CADP tools queries 

## 2. SVL queries

In [79]:
import pandas as pd
import numpy as np 

In [80]:
file_name = "9M-5-[1-2001]-[a]-lcm.bcg"
file_path = f"../graph/bcg/{file_name}"

In [81]:
before_promo_period = "2018-09-01"
promo_period ="2018-12-01"
after_promo_period = ":2019-02-01"

In [82]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.classes_ = np.load(f'../../plots/encoders/{file_name.replace("bcg","out")}.npy')
promo_products = pd.read_csv("../../datasets/Total/promo_products.csv")
promo_products = f':{":".join(promo_products.ARTICLE_ID.astype(str).unique())}:'

In [83]:
promo_products

':3425901014890:3425901019178:3425900000962:3011449262970:3425901029788:3425901015859:3425901015835:3425901014944:3425901018188:3425901018201:3425901027432:3425901027449:3011449300559:3425901018140:3267025011375:3267025017650:3267025010606:3267025001406:3267021013236:3267021013229:3267025000836:3267021030738:3267025007262:3267021030837:3267025007293:3267021030530:3267021020531:3267025007361:3267025007460:3267021020432:3267025013072:3267025013065:3267025000072:3011449257037:3068320110783:3425901019260:3425901037172:3425901025872:3425901048086:3425901017969:3425901028170:3425901018249:3425901090993:3011449260457:3425901028187:3425901026251:3425901019505:3425901018232:3425909001007:3425909002011:3425909002943:3425900000436:3425901018720:3425901027487:3425901018195:3011449014005:3425901029320:3425901019734:3425901027869:3425901017976:3425901029290:'

### 2.1 Adoption query

In [84]:
query_name = "product_adoption.svl"
query_path = f"../queries/svl/{query_name}"
file_path = f'../graph/bcg/9M-5-[1-2001]-[a]-lcm'

In [87]:
%%bash -s $file_path 
export PATH=$PATH:$CADP/bin.x64:$CADP/com
svl product_adoption.svl $1 ":2018-09-01" ":2018-12-01" ":2019-02-01"


property PRODUCT_ADOPTION
 | Product adoption

TRUE

[Customers] [Promotional products] Period

[101392, 103904, 105649, 105769, 111061, 116517, 117912, 123642, 123949, 125615, 128265, 130464, 131834, 132260, 137060, 138325, 145566, 146201, 147772, 149886, 152266, 153534, 153850, 155710, 157047, 160513, 165324, 171799, 172457, 173559, 178285, 181027, 181964, 184070, 184841, 188482, 189419, 189637, 191747, 195200, 197939, 202124, 203487, 207309, 211606, 213211, 214303, 215374, 220829, 225819, 226924, 228704, 236344, 238881, 241262, 241363, 245072, 249602, 252901, 257663, 262086, 263432, 267956, 269558, 270927, 271256, 274467, 274953, 281256, 285626, 287447, 288714, 290288, 291564, 294576, 295124, 296219, 297113, 298742, 302709, 303696, 304157, 307874, 310380, 313472, 317586, 318322, 319166, 321180, 326483, 326781, 329769, 339428, 341341, 343312, 343449, 344970, 349622, 354817, 355592, 363045, 365174, 366567, 367316, 368517, 372225, 372638, 376752, 380944, 381797, 387373, 388999, 400788

###  2.2 Loyalty query 

In [88]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl product_loyalty.svl $1 ":3461025:7461024:" ":2018-12-01" ":2019-02-01"


property product_loyalty

TRUE

[Customers] [Promotional products] Period

[1044697] [3461025] :2018-12-01
[211249] [7461024] :2018-12-01
[575459, 1044697, 1163180] [3461025] :2018-12-01
[575459] [3461025] :2018-12-01



### 2.3 Pomotion only query

In [108]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl promotion_only.svl $1 ":2018-09-01" ":2018-12-01" ":2019-02-01"


property promotion_only

TRUE

[Customers] [Promotional products] Period




In [109]:
!gedit promotion_only.svl


## Draft

In [6]:
text = """
    Hello, i'm trying here ${a} to write ${b}
"""
with open('temp.txt',"w") as file:
    file.write(text)
!cat temp.txt


    Hello, i'm trying here ${a} to write ${b}


In [8]:
%%bash
a=12 b=14 envsubst< temp.txt > query.txt

In [108]:
def run_query(query_path,graph_path,parameters,columns = ["user_id_x","itemsets_x","user_id_y","itemsets_y"],extra_cols =[],execution_time=False):
    """
        Run query using MCL and return the formated output
        Parameters
        ----
        query_path : str
        graph_path : str
        parameters : str
            ex: "T0=:2018-09-01 T1=:2018-12-01"
        columns : list of str
        extr_cols: list of str
            extra columns for queries having more return values than the default ones in columns 
    """
    !{parameters} envsubst < {query_path} > query.mcl
    data = !bcg_open {graph_path} evaluator4 query.mcl 
    data = data[2:-2]    
    data = [i.replace(":","").split() for i in data]
    data = pd.DataFrame(data,columns=columns+extra_cols)
    for i in columns+extra_cols:
        data[i] = data[i].apply(lambda x : set(x.split(",")))
    return format_query_output(data)

In [117]:
graph_path = '../graph/bcg/9M-5-\[1-2001\]-\[a\]-lcm.bcg'

# Short term loyalty 1 
parameters = "T0=:2018-09-01 T1=:2018-12-01"
query_path = "../queries/mcl/product_st_loyalty.mcl"
df  = run_query(query_path,graph_path,parameters)

# # Short term loyalty 2 
# parameters = "T0=:2018-12-01 T1=:2019-02-01"
# run_query(query_path,graph_path,parameters)

# #  Adoption  
# parameters = "T0=:2018-09-01 T1=:2018-12-01"
# query_path = "../queries/mcl/product_adoption.mcl"
# run_query(query_path,graph_path,parameters)

# # Promo exclusive 
# parameters = "T0=:2018-12-01 T1=:2019-02-01 T2=:2019-02-01"
# query_path = "../queries/mcl/promo_exclusive.mcl"
# run_query(query_path,graph_path,parameters,extra_cols=["user_id_z","itemsets_z"])

# # Product replacement 
# parameters = "T0=:2018-12-01 T1=:2019-02-01 T2=:2019-02-01"
# query_path = "../queries/mcl/replacement.mcl"
# run_query(query_path,graph_path,parameters,extra_cols=["user_id_z","itemsets_z"])

In [116]:
items = pd.read_csv("../../datasets/Total/items.csv",index_col=0)
items

,DESCRIPTION,IDS_LIST,ARTICLE_ID
0,1 6 RILLETTES THON 125G,[3019081236250],3019081236250
1,"1,5L SUMOL ORAN",[5601045000427],5601045000427
2,1/ BAGUETTE CLUB KAP,[3464381000276],3464381000276
3,1/2 BAG. VIENNOISE X2 170G,"[3760049790801, 3760049790818]",3760049790801
4,10 GOBELETS 27C,[3700232615702],3700232615702
...,...,...,...
11018,ZIGOH DESO FLEUR FUSHIA,[3700619604725],3700619604725
11019,ZIGOTO REGLISSE,[3103220009055],3103220009055
11020,ZIP AMIS LUDENDO,[3700300542008],3700300542008
11021,ZIP POUPEE LAVANDE,[3453131039413],3453131039413


In [ ]:
#### adoptions = {}
df["user"] = df.apply(lambda x : x["user_id_x"].intersection(x["user_id_y"]),axis=1)
df

In [139]:
df.user

,user_id_x,itemsets_x,user_id_y,itemsets_y,user_id_z
0,"{263432, 252901, 1162329, 410397, 141196, 3211...",{65504175},"{116517, 436321, 208325, 492488, 834995, 47372...",{4175},"{410397, 321180, 263432, 252901}"
1,"{1184015, 252901, 339822, 444167, 321180, 1979...",{65494175},"{116517, 436321, 208325, 492488, 834995, 47372...",{4175},"{321180, 197939, 252901}"
2,"{1043083, 836821, 282802, 1061200, 606741, 418...",{41474175},"{116517, 436321, 208325, 492488, 834995, 47372...",{4175},"{836821, 339428, 1061200, 606741, 563868, 4007..."
3,"{897154, 102136, 241262, 430600, 873625, 189637}",{41854175},"{116517, 436321, 208325, 492488, 834995, 47372...",{4175},"{873625, 241262, 189637, 430600}"
4,"{897154, 102136, 241262, 873625, 189637}",{418541804175},"{116517, 436321, 208325, 492488, 834995, 47372...",{4175},"{873625, 241262, 189637}"
5,"{387373, 380944, 195200, 463969, 579243, 46846...",{62644175},"{116517, 436321, 208325, 492488, 834995, 47372...",{4175},"{387373, 380944, 195200, 579243, 463969, 51102..."
6,"{480899, 517175, 1022740, 102136, 195246, 3673...",{29834175},"{116517, 436321, 208325, 492488, 834995, 47372...",{4175},"{517175, 1022740, 367316, 1029030, 189419, 415..."
7,"{313887, 458921, 150817, 477520, 267912, 48235...",{29944175},"{116517, 436321, 208325, 492488, 834995, 47372...",{4175},"{482352, 458921, 543430, 477520}"
8,"{311494, 516054, 600652, 368001, 896797, 10805...",{4186},"{665395, 383984, 561007, 144596, 747498, 10805...",{4186},"{1080578, 228538, 447321, 459305, 484507, 1033..."
9,"{311494, 600652, 1080578, 623497, 484507, 2706...",{41864160},"{665395, 383984, 561007, 144596, 747498, 10805...",{4186},"{1080578, 470752, 908982, 407466, 348948, 6221..."
